# Fitting to the experimental data

In this notebook we will show how to load a CIF file, an experimental profile and how to perform a parameter fit.


#### Import Python packages

In [ ]:
# esyScience, technique-independent
from easyCore import np
from easyCore.Fitting.Fitting import Fitter

# esyScience, diffraction
from easyDiffractionLib import Phases
from easyDiffractionLib.sample import Sample as Job
from easyDiffractionLib.interface import InterfaceFactory as Calculator
from easyDiffractionLib.Elements.Experiments.Pattern import Pattern1D
from easyDiffractionLib.Elements.Backgrounds.Point import PointBackground, BackgroundPoint
from easyDiffractionLib.Profiles.P1D import Instrument1DCWParameters

# Vizualization
import py3Dmol
import matplotlib.pyplot as plt

## --- Sample ---

#### Show a CIF file content

In [ ]:
cif_fname = 'PbSO4.cif'

with open(cif_fname, 'r') as f:
    content = f.read()
    
print(content)

#### Load structure from a CIF file

In [ ]:
phases = Phases.from_cif_file(cif_fname)
phase = phases[0]

print(phases)
print(phase)

#### Visualise the structure

In [ ]:
structure = py3Dmol.view()
structure.addModel(phase.to_cif_str(), 'cif')
structure.setStyle({'sphere':{'colorscheme':'Jmol','scale':.2},'stick':{'colorscheme':'Jmol','radius': 0.1}})
structure.addUnitCell()
structure.replicateUnitCell(2,2,1)
structure.zoomTo()

## --- Experiment ---

#### Show measured data as text

In [ ]:
meas_fname = 'D1A@ILL.xye'

with open(meas_fname, 'r') as f:
    content = f.read()
    
# print(content)

#### Load the measured data

In [ ]:
meas_x, meas_y, meas_e = np.loadtxt(meas_fname, unpack=True)

#### Visualize the measured data

In [ ]:
%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.legend()

## --- Analysis ---

#### Create job with default parameters for the 1D powder neutron diffraction experiment with constant wavelength 

In [ ]:
calculator = Calculator()

print(f"Current calculator engine: {calculator.current_interface_name}")

In [ ]:
job = Job(phases=phases, parameters=Instrument1DCWParameters.default(), calculator=calculator)

#### Generate the calculated data

In [ ]:
calc_y_cryspy = calculator.fit_func(meas_x)

#### Visualize both the measured and calculated data

In [ ]:
%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.legend()

#### Set scale manually

In [ ]:
job.pattern.scale = 100

In [ ]:
calc_y_cryspy = calculator.fit_func(meas_x)

%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.legend()

#### Set wavelength manually

In [ ]:
job.parameters.wavelength = 1.912

In [ ]:
calc_y_cryspy = calculator.fit_func(meas_x)

%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.legend()

#### Set background points manually

In [ ]:
bkg = PointBackground(linked_experiment='PbSO4')

bkg.append(BackgroundPoint.from_pars(meas_x[0], 200))
bkg.append(BackgroundPoint.from_pars(meas_x[-1], 250))

job.set_background(bkg)

In [ ]:
calc_y_cryspy = calculator.fit_func(meas_x)

%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.legend()

#### Define parameters to optimize

In [ ]:
job.pattern.scale.fixed = False
job.pattern.zero_shift.fixed = False
job.parameters.resolution_u.fixed = False
job.parameters.resolution_v.fixed = False
job.parameters.resolution_w.fixed = False
job.backgrounds[0][0].y.fixed = False
job.backgrounds[0][1].y.fixed = False

In [ ]:
print(job.pattern.scale)
print(job.pattern.zero_shift)
print(job.parameters.resolution_u)
print(job.parameters.resolution_v)
print(job.parameters.resolution_w)
print(job.backgrounds[0][0])
print(job.backgrounds[0][1])

#### Initalize the fitting engine and perform the fit

In [ ]:
fitter = Fitter(job, calculator.fit_func)

print(f"Available minimizers: {fitter.available_engines}")
print(f"Current minimizer: {fitter.current_engine.name}")
print(f"Available methods of current minimizers: {fitter.available_methods()}")

In [ ]:
result = fitter.fit(meas_x, meas_y, weights=1/meas_e, method='least_squares')

print("The fit has been successful: {}".format(result.success))
if result.success:    
    print("The gooodness of fit (chi2) is: {}".format(result.reduced_chi))
    print(job.pattern.scale)
    print(job.pattern.zero_shift)
    print(job.parameters.resolution_u)
    print(job.parameters.resolution_v)
    print(job.parameters.resolution_w)
    print(job.backgrounds[0][0])
    print(job.backgrounds[0][1])

In [ ]:
calc_y_cryspy = calculator.fit_func(meas_x)

%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.plot(meas_x, meas_y-calc_y_cryspy, label='Imeas-Icalc (CrysPy)')
plt.legend()

#### Change calculator engine to CrysFML

In [ ]:
job.interface.switch('CrysFML')
job.update_bindings()

print(f"Current calculator engine: {job.interface.current_interface_name}")
print(f"Current minimizer: {fitter.current_engine.name}")

#### Show results of both CrysPy and CrysFML calculations (before fitting)

In [ ]:
calc_y_crysfml = calculator.fit_func(meas_x)

%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.plot(meas_x, calc_y_crysfml, label='Icalc (CrysFML)')
plt.legend()

#### Perform the fit with CrysFML

In [ ]:
result = fitter.fit(meas_x, meas_y, weights=1/meas_e, method='least_squares')

print("The fit has been successful: {}".format(result.success))
if result.success:    
    print("The gooodness of fit (chi2) is: {}".format(result.reduced_chi))
    print(job.pattern.scale)
    print(job.pattern.zero_shift)
    print(job.parameters.resolution_u)
    print(job.parameters.resolution_v)
    print(job.parameters.resolution_w)
    print(job.backgrounds[0][0])
    print(job.backgrounds[0][1])

#### Show results of both CrysPy and CrysFML calculations (after fitting)

In [ ]:
calc_y_crysfml = calculator.fit_func(meas_x)

%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.plot(meas_x, calc_y_crysfml, label='Icalc (CrysFML)')
plt.legend()